В этом ноутбуке я использую cling, C++ kernel for Jupyter Notebook. Обычно Jupyter Notebook используется для интерактивных экспериментов на питоне или R, но в CERN сделали возможность писать здесь на C++.
Устанавливал вот по этой инструкции: http://shuvomoy.github.io/blog/programming/2016/08/04/Cpp-kernel-for-Jupyter.html.

Здесь можно делать то, чего нельзя в обычной программе на C++: не иметь main и исполнять код вне функций. Это удобно для демонстрационных целей.

# Генерация случайных чисел

Бывает так, что в алгоритме хочется как-то использовать случайные величины. Например, хочется случайным образом перемешать массив, или сгенерировать случайный тест, или сгенерировать случайный пароль.

Проблема в том, что компьютер — это штука для исполнения детерминированных алгоритмов. Процессор исполняет инструкции согласно строгой спецификации, и обычно не в состоянии взять откуда-то случайное число и положить его в память. Даже если попытаться не инициализировать память или регистр, а потом оттуда прочитать, скорее всего, то, что мы прочитаем, будет как-то связано с тем, что туда когда-то писалось. В обычной модели компьютера просто неоткуда взять и изобрести по-честному случайную величину, не используя какие-нибудь устройства ввода.

На самом деле, устройств ввода у ПК много. Ядро Linux умеет генерировать по-честному случайные величины, анализируя для этого временные ряды использования клавиатуры и мыши и прерываний процессора. Ещё можно было бы использовать шум в микрофоне или веб-камере. Проблема такой честной генерации в том, что эти данные обновляются очень редко по сравнению с частотой, с которой процессор исполняет инструкции. Поэтому ядро поддерживает так называемый entropy pool. Он постепенно накапливает энтропию, то есть "честность случайности", пользуясь всем, чем можно, и расходует её всегда, когда процессы просят у ядра настоящие случайные значения. Если энтропия заканчивается, процесс может зависнуть, дожидаясь ответа от ядра. Криптографическая библиотека OpenSSL может в этой ситуации переключиться на менее честную генерацию, что менее безопасно. Это вполне возможная ситуация на серверах, которые работают в датацентрах со стабильным климатом и без подключенных клавиатуры, мыши, микрофона и вебкамеры.

Возможно, вы когда-то подготавливали rsa-ключ программой PuTTY, чтобы пользоваться ssh или git на удалённом сервере без пароля, и PuTTY просил довольно долго водить мышкой, чтобы набрать энтропию. Вот это оно.

В общем, получить много по-честному случайных чисел из внешней среды сложно и потенциально долго. Поэтому практичный подход — генерировать *псевдо*случайную последовательность с помощью какого-нибудь специального алгоритма. *Псевдо*случайность значит, что алгоритм на самом деле детерминированный, то есть если его инициализировать одинаковым образом, то и сгенерированная последовательность будет одинаковая. Но при этом у хороших алгоритмов последовательность выглядит и проходит всякие тесты, почти как по-настоящему случайная. Но, конечно, до некоторых пор: если алгоритм использует конечное количество памяти, чтобы хранить свою позицию в последовательности, рано или поздно он придёт в состояние, в котором уже был, и тогда последовательность станет периодической, чего не бывает со случайными последовательностями.

## minstd

До середины девяностых был особенно популярен довольно простой алгоритм генерации псевдослучайных чисел. У него даже есть устойчивое название — minstd. Выглядит он следующим образом.

Пусть $\kappa_{i-1}$ — число, которое мы сгенерировали в прошлый раз. Если ещё не генерировали, то это какое-нибудь число, которое надо взять из внешнего источника, обычно его называют зерном (seed) рандома.

Тогда следующее число найдём по формуле: $\kappa_{i} = g \cdot \kappa_{i-1} \mod M$, где $g$ и $M$ — заранее фиксированные аккуратно подобранные числа.

Всё. https://en.wikipedia.org/wiki/Lehmer_random_number_generator

In [1]:
#include <iostream>

In [2]:
class MyMinstd {
public:
    // M и G — статические поля. Это значит, что они хранятся не в каждом экземпляре класса, а
    // в единственном месте в памяти программы, как глобальные переменные.
    // Но при этом они находятся в пространстве видимости класса:
    // - публичные статические поля доступны снаружи доступны только с явным указанием класса, например,
    //   MyMinstd::G;
    // - приватные статические поля недоступны снаружи так же, как не статические.
    //
    // Внутри чисел можно писать апострофы для удобства чтения. Компилятор их игнорирует.
    static const long long M = 2'147'483'647;
    static const long long G = 48'271;

    explicit MyMinstd(int seed = 1)
        : state(seed)
    {}
    
    int operator()() {
        state = (state * G) % M;
        return state;
    }
    
private:
    int state;
};

In [3]:
MyMinstd myGenerator11(11);
std::cout << myGenerator11() << '\n';
std::cout << myGenerator11() << '\n';
std::cout << myGenerator11() << '\n';
std::cout << myGenerator11() << '\n';

530981
2008663734
1320441864
1734574184


(std::basic_ostream<char, std::char_traits<char> > &) @0x7f2f16963e20


In [4]:
template<class Gen>
void show(Gen gen, int n) {
    for (int i = 0; i < n; ++i) {
        std::cout << gen() % 100 << ' ';
    }
    std::cout << '\n';
}

In [5]:
show(MyMinstd(42), 20);
show(MyMinstd(90320905), 10);

82 7 37 15 42 57 75 58 5 45 41 13 46 60 18 51 6 21 56 10 
45 41 13 46 60 18 51 6 21 56 


(void) @0x7f2f10a2ec80


В стандартной библиотеке C++ есть minstd:

In [6]:
#include <random>

In [7]:
show(std::minstd_rand(42), 20);

82 7 37 15 42 57 75 58 5 45 41 13 46 60 18 51 6 21 56 10 


(void) @0x7f2f10a2ec80


Качество алгоритмов такого вида широко раскритиковано, особенно [некоторых](https://en.wikipedia.org/wiki/RANDU), зато они очень простые и быстрые.

## `rand()`

Алгоритм, который должна реализовать функция rand(), унаследованная в C++ из C, не зафиксирован никаким стандартом. В реализации glibc, используемой обычно под linux, видимо, сделано что-то похожее на minstd: [link](https://www.mathstat.dal.ca/~selinger/random/).

Использовать эту функцию в C++ крайне не рекомендуется, потому что есть лучшие альтернативы.

* Так как алгоритм не зафиксирован, она буквально может всё время возвращать одно и то же [число 4](https://xkcd.com/221/). Согласно [cppreference](http://ru.cppreference.com/w/cpp/numeric/random/rand), *Нет никаких гарантий в отношении криптографической стойкости сгенерированных случайных чисел. В прошлом, в некоторых реализациях rand() имели место серьезные недостатки случайного распределения чисел (к примеру, единицы в нижних разрядах между вызовами просто чередовались 1-0-1-0-...).*. Особенно если [брать результат по модулю](https://stackoverflow.com/questions/14678957/libc-random-number-generator-flawed).

* Кроме того, она имеет глобальное состояние, то есть во всей вашей программе может одновременно генерироваться только одна псевдослучайная последовательность. Нельзя в одной части погенерировать чисел так, чтобы на другой, совсем не связанной с первой, части программы, это не отразилось, если, конечно, не поменять зерно.

* Функция может работать вовсе некорректно, если в вашей программе несколько потоков исполнения.

## Что стоит использовать

Сейчас в моде алгоритм, который называется Mersenne Twister. Он качественный и у него очень большой период, до $2^{19937} − 1$. В стандартной библиотеке Python он [используется](https://docs.python.org/2.7/library/random.html) по умолчанию начиная с версии 2.3.

Использовать его в C++ так же легко, как std::minstd_rand или MyMinstd:

In [8]:
#include <random>

In [9]:
std::mt19937 twister(42);

std::cout << twister() << '\n';
std::cout << twister() << '\n';

show(twister, 20);

1608637542
3421126067
76 14 26 35 20 24 50 13 78 14 10 54 31 72 15 95 67 6 49 76 


(void) @0x7f2f10a2ec80


std::mt19937 генерирует по 32 бита за раз. Если хочется сразу по 64 бита, можно воспользоваться другой вариацией того же класса:

In [10]:
std::mt19937_64 twister64(42);
twister64()

(unsigned long) 13930160852258120406


При этом нужно понимать, что у него довольно большое состояние по сравнению с другими алгоритмами:

In [11]:
sizeof (std::mt19937)

(unsigned long) 5000


In [12]:
sizeof (std::minstd_rand)

(unsigned long) 8


Это в байтах. Поэтому тут как с вектором, если вы передаёте генератор в функцию, и вам не нужна копия, передавайте по ссылке. 

## Распределения

Скорее всего, вам нужны не просто случайные 32 или 64 бита, а число из какого-то распределения. Скажем, нужно равновероятно выбрать номер элемента в массиве или равновероятно выбрать вещественное число от -1 до 1, или с нормальным распределением выбрать вещественное число в окрестности 0, и т. д. Всё это можно сделать, набирая случайные 32 или 64 бита и преобразовывая их по каким-нибудь формулам, но ошибиться при этом легче, чем не ошибиться.

Поэтому в стандартной библиотеке есть классы, моделирующие многие одномерные распределения. Это тоже именно классы, потому что обычно распределения имеют параметры: границы отрезка, из которого должны равновероятно браться числа, или медиана и дисперсия для нормального распределения. Эти параметры запоминаются, и затем если передавать в operator() какой-нибудь генератор случайных чисел, то можно получать уже числа из распределения.

In [13]:
std::uniform_int_distribution<int> uniformInt(0, 9);
std::cout << uniformInt(twister) << '\n';
std::cout << uniformInt(twister) << '\n';
std::cout << uniformInt(twister) << '\n';
std::cout << uniformInt(twister) << '\n';

9
1
7
7


(std::basic_ostream<char, std::char_traits<char> > &) @0x7f2f16963e20


In [14]:
#include <vector>

In [15]:
template<class Distr, class Gen>
void showDistr(Distr distr, Gen& gen, int n) {
    std::vector<int> count(n, 0);
    for (int i = 0; i < 100000; ++i) {
        int randValue = distr(gen);
        ++count[randValue];
    }
    for (auto c : count) {
        std::cout << c << ' ';
    }
    std::cout << '\n';
    for (int i = 0; i < n; ++i) {
        std::cout << i << ": ";
        for (int j = 0; j < count[i] / 150; ++j) {
            std::cout << '*';
        }
        std::cout << '\n';
    }
}

In [16]:
showDistr(std::uniform_int_distribution<int>(0, 19), twister, 20);

4967 4977 5033 4939 4945 5095 5150 4953 4987 4958 5064 5008 5001 5073 5007 4928 5000 4959 4996 4960 
0: *********************************
1: *********************************
2: *********************************
3: ********************************
4: ********************************
5: *********************************
6: **********************************
7: *********************************
8: *********************************
9: *********************************
10: *********************************
11: *********************************
12: *********************************
13: *********************************
14: *********************************
15: ********************************
16: *********************************
17: *********************************
18: *********************************
19: *********************************


(void) @0x7f2f10a2ec80


In [17]:
showDistr(std::binomial_distribution<int>(49, 0.420), twister, 50);

0 0 0 0 0 0 0 5 12 30 93 228 499 1004 1941 3178 4881 6870 8822 10291 11530 11365 10400 9016 7002 5105 3381 2002 1189 647 289 131 61 16 7 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0: 
1: 
2: 
3: 
4: 
5: 
6: 
7: 
8: 
9: 
10: 
11: *
12: ***
13: ******
14: ************
15: *********************
16: ********************************
17: *********************************************
18: **********************************************************
19: ********************************************************************
20: ****************************************************************************
21: ***************************************************************************
22: *********************************************************************
23: ************************************************************
24: **********************************************
25: **********************************
26: **********************
27: *************
28: *******
29: ****
30: *
31: 
32: 
33: 
34: 
35: 
36: 
37: 

(void) @0x7f2f10a2ec80


In [18]:
showDistr(std::bernoulli_distribution(0.228), twister, 2);

77106 22894 
0: **********************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************
1: ********************************************************************************************************************************************************


(void) @0x7f2f10a2ec80


std::uniform_int_distribution мог бы быть реализован как-то так:

In [19]:
template<class T>
class MyUniform {
public:
    explicit MyUniform(T lower, T upper)
        : lower(lower)
        , upper(upper)
    {}
    
    template<class Gen>
    T operator()(Gen& gen) {
        return lower + gen() % (upper - lower + 1);
    }
    
private:
    T lower, upper;
};

In [20]:
showDistr(MyUniform<int>(3, 22), twister, 25);

0 0 0 4971 5022 5183 4991 4913 5066 5029 4966 4961 5076 5057 4999 4957 4877 5103 5136 4849 4939 4996 4909 0 0 
0: 
1: 
2: 
3: *********************************
4: *********************************
5: **********************************
6: *********************************
7: ********************************
8: *********************************
9: *********************************
10: *********************************
11: *********************************
12: *********************************
13: *********************************
14: *********************************
15: *********************************
16: ********************************
17: **********************************
18: **********************************
19: ********************************
20: ********************************
21: *********************************
22: ********************************
23: 
24: 


(void) @0x7f2f10a2ec80


## Настоящий рандом

Кроме генераторов псевдослучайных чисел, в C++ есть класс с похожим интерфейсом, который пытается, если это возможно, получить от системы настоящее случайное число:

In [21]:
std::random_device trueRandom;

In [22]:
trueRandom()

(unsigned int) 3189666475


In [23]:
trueRandom()

(unsigned int) 2514577865


In [24]:
showDistr(MyUniform(0, 19), trueRandom, 20);

4962 5061 4918 4983 5064 4945 4881 5080 4977 4930 5008 5001 4943 5059 5131 5057 4906 5083 4985 5026 
0: *********************************
1: *********************************
2: ********************************
3: *********************************
4: *********************************
5: ********************************
6: ********************************
7: *********************************
8: *********************************
9: ********************************
10: *********************************
11: *********************************
12: ********************************
13: *********************************
14: **********************************
15: *********************************
16: ********************************
17: *********************************
18: *********************************
19: *********************************


(void) @0x7f2f10a2ec80


Использовать его надо понемножку (не так, как я сейчас, быстро набрав $10^5$ чисел), чтобы не наступило истощение entropy pool. В примерах на cppreference из random_device берут зерно для mt19937.

## Further reading

- https://en.wikipedia.org/wiki/Entropy_%28computing%29

- https://www.atlasobscura.com/places/encryption-lava-lamps — У Cloudflare есть *ферма лавовых ламп* для получения настоящего рандома.

- http://en.cppreference.com/w/cpp/numeric/random

- https://ru.wikipedia.org/wiki/Генератор_псевдослучайных_чисел

- https://ru.wikipedia.org/wiki/Тасование_Фишера_—_Йетса, http://ru.cppreference.com/w/cpp/algorithm/random_shuffle — равновероятное перемешивание массива за O(n), обратите внимание, что std::random_shuffle устарел и уже удалён из последнего стандарта C++, потому что использует std::rand(). Вместо него можно использовать std::shuffle.

- https://ru.wikipedia.org/wiki/Reservoir_sampling — выбор случайного сочетания фиксированного размера k из потока длины n за O(n) времени и всего O(k) памяти.

- https://ru.wikipedia.org/wiki/Преобразование_Бокса_—_Мюллера — как имея равномерное распределение получить нормальное.